# 🚀️ Scene Index: Advanced Visual Search

<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/guides/scene-index/advanced_visual_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


> This guide assumes you are already familiar with the concept of Scene Indexing. If you are not, please refer to our [Scene Index: QuickStart](https://github.com/video-db/videodb-cookbook/blob/main/guides/video/scene-index/quickstart.ipynb) Guide to get up to speed.  


We provide you with easy-to-use objects and functions to bring flexibility in designing your visual understanding pipeline. With these tools, you have the freedom to..

1. Extract scene according to your use case.
2. Go to frame level abstraction. 
3. Assign label, custom model description for each frame. 
4. Use of multiple models, prompts for each scene or frame to convert information to text. 
5. Send multiple frames to vision model for better temporal activity understanding.

## Setup
---

### 📦  Installing packages   

In [ ]:
!pip install videodb

### 🔑 API Keys

In [ ]:
import videodb
import os
from getpass import getpass

api_key = getpass("Please enter your VideoDB API Key: ")

os.environ["VIDEO_DB_API_KEY"] = api_key

### 🌐 Connect to VideoDB

In [3]:
from videodb import connect

conn = connect()
coll = conn.get_collection()

### 🎥  Upload Video

In [4]:
video = coll.upload(url="https://www.youtube.com/watch?v=LejnTJL173Y")

## ✂️🎬 Extract Scenes
---

In [5]:
from videodb import SceneExtractionType

scene_collection = video.extract_scenes(
    extraction_type=SceneExtractionType.time_based,
    extraction_config={"time": 30, "select_frames": ["middle"]},
)

In [ ]:
print("This is scene collection id", scene_collection.id)
print("This is scene collection config", scene_collection.config)

**Note**: Image Upload might take time ranging from 5s-60s. Re-fetch the scene collection if image_url is None

In [ ]:
scene_collection = video.get_scene_collection(scene_collection.id)
print(scene_collection.scenes[0].frames[0].url)


## 📝 Annotating Scene
---

In [ ]:
# get scene from collection
scenes = scene_collection.scenes

def describe_images(urls, start, end):
    # Plug your VLLM or CV model here
    return f"This is a scene from {start}s-{end}s" 

for scene in scenes:
  print(f"Scene Duration {scene.start}-{scene.end}")

  frames_url = [frame.url for frame in scene.frames]
  scene.description = describe_images(frames_url, start=scene.start, end=scene.end)

## 📝 Annotating Frames 
---

In [ ]:
def describe_image(url, time):
    # Plug your VLLM or CV model here
    return f"This is a keyframe at {time}s" 


# Iterate through each scene
for scene in scenes:
    
    # Iterate through each frame in the scene
    for frame in scene.frames:
        print(f"Frame at {frame.frame_time} {frame.url}")
        frame.description = describe_image(frame.url, time=frame.frame_time) 

## Indexing the Annotated Scenes
---

In [ ]:
index_id = video.index_scenes(scenes=scenes, name="My Custom Annotation")

scene_index = video.get_scene_index(index_id)
print(scene_index)

## Searching the Index
---

> Note: it might take a additional 5-10 seconds for your index to become available for search

In [ ]:
from videodb import IndexType

# search using the index_id
res = video.search(query="first 29s", index_type=IndexType.scene, index_id=index_id)
res.play()

## 🧑‍💻 Next Steps
---


Check out the other resources and tutorials using Scene Indexing
* If you want to bring your own scene descriptions and annotations, explore the [Custom Annotations  Pipeline](https://github.com/video-db/videodb-cookbook/blob/main/guides/video/scene-index/custom_annotations.ipynb)
* Experiment with extraction algorithms, prompts, and search using the [Playground for Scene Extractions](https://github.com/video-db/videodb-cookbook/blob/main/guides/video/scene-index/playground_scene_extraction.ipynb)

If you have any questions or feedback. Feel free to reach out to us 🙌🏼

* [Discord](https://colab.research.google.com/corgiredirector?site=https%3A%2F%2Fdiscord.gg%2Fpy9P639jGz)
* [GitHub](https://github.com/video-db)
* [VideoDB](https://colab.research.google.com/corgiredirector?site=https%3A%2F%2Fvideodb.io)
* [Email](ashu@videodb.io)